In [1]:
# modified from: https://github.com/thstkdgus35/EDSR-PyTorch
import math
from argparse import Namespace
import torch
import torch.nn as nn
import torch.nn.functional as F

from models import register


def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size,
        padding=(kernel_size // 2), bias=bias)


class ResBlock(nn.Module):
    def __init__(
            self, conv, n_feats, kernel_size,
            bias=True, bn=False, act=nn.ReLU(True), res_scale=1):

        super(ResBlock, self).__init__()
        m = []
        for i in range(2):
            m.append(conv(n_feats, n_feats, kernel_size, bias=bias))
            if bn:
                m.append(nn.BatchNorm2d(n_feats))
            if i == 0:
                m.append(act)

        self.body = nn.Sequential(*m)
        self.res_scale = res_scale

    def forward(self, x):
        res = self.body(x).mul(self.res_scale)
        res += x

        return res


class Upsampler(nn.Sequential):
    def __init__(self, conv, scale, n_feats, bn=False, act=False, bias=True):

        m = []
        if (scale & (scale - 1)) == 0:
            for _ in range(int(math.log(scale, 2))):
                m.append(conv(n_feats, 4 * n_feats, 3, bias))
                m.append(nn.PixelShuffle(2))
                if bn:
                    m.append(nn.BatchNorm2d(n_feats))
                if act == 'relu':
                    m.append(nn.ReLU(True))
                elif act == 'prelu':
                    m.append(nn.PReLU(n_feats))

        elif scale == 3:
            m.append(conv(n_feats, 9 * n_feats, 3, bias))
            m.append(nn.PixelShuffle(3))
            if bn:
                m.append(nn.BatchNorm2d(n_feats))
            if act == 'relu':
                m.append(nn.ReLU(True))
            elif act == 'prelu':
                m.append(nn.PReLU(n_feats))
        else:
            raise NotImplementedError

        super(Upsampler, self).__init__(*m)


class MeanShift(nn.Conv2d):
    def __init__(
            self, rgb_range,
            rgb_mean=(0.4488, 0.4371, 0.4040), rgb_std=(1.0, 1.0, 1.0), sign=-1):
        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        std = torch.Tensor(rgb_std)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1) / std.view(3, 1, 1, 1)
        self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean) / std
        for p in self.parameters():
            p.requires_grad = False

@register('edsr-baseline')
class EDSR(nn.Module):
    def __init__(self, args, conv=default_conv):
        super(EDSR, self).__init__()
        self.args = args
        n_resblocks = args.n_resblocks
        n_feats = args.n_feats
        kernel_size = 3
        scale = args.scale[0]
        act = nn.ReLU(True)

        # define head module
        m_head = [conv(args.n_colors, n_feats, kernel_size)]

        # define body module
        m_body = [
            ResBlock(
                conv, n_feats, kernel_size, act=act, res_scale=args.res_scale
            ) for _ in range(n_resblocks)
        ]
        m_body.append(conv(n_feats, n_feats, kernel_size))

        self.head = nn.Sequential(*m_head)
        self.body = nn.Sequential(*m_body)

        if args.no_upsampling:
            self.out_dim = n_feats
        else:
            self.out_dim = args.n_colors
            # define tail module
            m_tail = [
                Upsampler(conv, scale, n_feats, act=False),
                conv(n_feats, args.n_colors, kernel_size)
            ]
            self.tail = nn.Sequential(*m_tail)

        self.sub_mean = MeanShift(args.rgb_range)
        self.add_mean = MeanShift(args.rgb_range, sign=1)

        if args.pretrained_path:
            try:
                pretrained_dict = torch.load(args.pretrained_path)
                self.load_state_dict(pretrained_dict)
                print("Pretrained model loaded successfully.")
            except FileNotFoundError:
                print(f"File not found: {args.pretrained_path}")
            except RuntimeError as e:
                print(f"Error loading model: {e}")

    def forward(self, x):
        x = self.head(x)
        res = self.body(x)
        res += x

        if self.args.no_upsampling:
            x = res
        else:
            x = self.tail(res)
        return x

    def load_state_dict(self, state_dict, strict=True):
        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name in own_state:
                if isinstance(param, nn.Parameter):
                    param = param.data
                try:
                    own_state[name].copy_(param)
                except RuntimeError:
                    if name.find('tail') == -1:
                        raise RuntimeError('While copying the parameter named {}, '
                                           'whose dimensions in the model are {} and '
                                           'whose dimensions in the checkpoint are {}.'
                                           .format(name, own_state[name].size(), param.size()))
            elif strict:
                if name.find('tail') == -1:
                    raise KeyError('unexpected key "{}" in state_dict'
                                   .format(name))


class SegmentationHead(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size=3, upsampling=1):
        conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        upsampling = nn.UpsamplingBilinear2d(scale_factor=upsampling) if upsampling > 1 else nn.Identity()
        super().__init__(conv2d, upsampling)

@register("encoder")
class Encoder(nn.Module):
    def __init__(self, args, n_class=100):
        super(Encoder, self).__init__()
        self.args = args
        self.edsr = EDSR(args)
        self.segmentation_head = SegmentationHead(in_channels=args.n_feats, out_channels=n_class)
        self.out_dim = args.n_feats

    def forward(self, x):
        x = self.edsr(x)
        logits = self.segmentation_head(x)
        B, Class, H, W = logits.shape
        logits = logits.permute(0, 2, 3, 1).contiguous().view(B * H * W, Class)
        if self.training:
            logits = F.gumbel_softmax(logits, tau=1, hard=False)
        if not self.training:
            logits = F.gumbel_softmax(logits, tau=1, hard=True)
        logits = logits.view(B, H, W, Class).permute(0, 3, 1, 2).contiguous()
        return x, logits


@register('edsr-baseline')
def make_encoder_baseline(n_resblocks=16, n_feats=64, res_scale=1, scale=2, no_upsampling=True, rgb_range=1, n_class=100):
    url = {
        'r16f64x2': 'models/weights/edsr_baseline_x2-1bc95232.pt',
        'r16f64x3': 'models/weights/edsr_baseline_x3-abf2a44e.pt',
        'r16f64x4': 'models/weights/edsr_baseline_x4-6b446fab.pt',
        'r32f256x2': 'models/weights/edsr_x2-0edfb8a3.pt',
        'r32f256x3': 'models/weights/edsr_x3-ea3ef2c6.pt',
        'r32f256x4': 'models/weights/edsr_x4-4f62e9ef.pt'
    }
    url_name = 'r{}f{}x{}'.format(n_resblocks, n_feats, scale)
    if url_name in url:
        url = url[url_name]
    else:
        url = None
    args = Namespace()
    args.n_resblocks = n_resblocks
    args.n_feats = n_feats
    args.res_scale = res_scale
    args.scale = [scale]
    args.no_upsampling = no_upsampling
    args.rgb_range = rgb_range
    args.n_colors = 3
    args.pretrained_path = url
    return Encoder(args, n_class)


@register('edsr-large')
def make_encoder_large(n_resblocks=32, n_feats=256, res_scale=0.1, scale=2, no_upsampling=True, rgb_range=1, n_class=100):
    url = {
        'r16f64x2': 'models/weights/edsr_baseline_x2-1bc95232.pt',
        'r16f64x3': 'models/weights/edsr_baseline_x3-abf2a44e.pt',
        'r16f64x4': 'models/weights/edsr_baseline_x4-6b446fab.pt',
        'r32f256x2': 'models/weights/edsr_x2-0edfb8a3.pt',
        'r32f256x3': 'models/weights/edsr_x3-ea3ef2c6.pt',
        'r32f256x4': 'models/weights/edsr_x4-4f62e9ef.pt'
    }
    url_name = 'r{}f{}x{}'.format(n_resblocks, n_feats, scale)
    if url_name in url:
        url = url[url_name]
    else:
        url = None
    args = Namespace()
    args.n_resblocks = n_resblocks
    args.n_feats = n_feats
    args.res_scale = res_scale
    args.scale = [scale]
    args.no_upsampling = no_upsampling
    args.rgb_range = rgb_range
    args.n_colors = 3
    args.pretrained_path = url
    return Encoder(args, n_class)


if __name__ == '__main__':
    network = make_encoder_baseline()
    a = torch.rand(1, 3, 48, 48)
    print(network(a)[0].shape, network(a)[1].shape)


Registered models: dict_keys([])
File not found: models/weights/edsr_baseline_x2-1bc95232.pt
torch.Size([1, 64, 48, 48]) torch.Size([1, 100, 48, 48])


C:\Users\pook0\AppData\Local\Temp\ipykernel_4288\3632505019.py:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(args.pretrained_path)


In [37]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import models
from models import register

from utils import to_pixel_samples


def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size // 2), bias=bias)


def make_coord(shape, ranges=None, flatten=True):
    """ Make coordinates at grid centers."""
    coord_seqs = []
    for i, n in enumerate(shape):
        if ranges is None:
            v0, v1 = -1, 1
        else:
            v0, v1 = ranges[i]
        r = (v1 - v0) / (2 * n)
        seq = v0 + r + (2 * r) * torch.arange(n).float()
        coord_seqs.append(seq)
    ret = torch.stack(torch.meshgrid(*coord_seqs), dim=-1)
    if flatten:
        ret = ret.view(-1, ret.shape[-1])
    return ret


def generate_meshgrid(height, width):
    """
    Generate a meshgrid of coordinates for a given image dimensions.
    Args:
        height (int): Height of the image.
        width (int): Width of the image.
    Returns:
        torch.Tensor: A tensor of shape [height * width, 2] containing the (x, y) coordinates for each pixel in the image.
    """
    # Generate all pixel coordinates for the given image dimensions
    y_coords, x_coords = torch.arange(0, height), torch.arange(0, width)
    # Create a grid of coordinates
    yy, xx = torch.meshgrid(y_coords, x_coords)
    # Flatten and stack the coordinates to obtain a list of (x, y) pairs
    all_coords = torch.stack([xx.flatten(), yy.flatten()], dim=1)
    return all_coords


def fetching_features_from_tensor(image_tensor, input_coords):
    """
    Extracts pixel values from a tensor of images at specified coordinate locations.
    Args:
        image_tensor (torch.Tensor): A 4D tensor of shape [batch, channel, height, width] representing a batch of images.
        input_coords (torch.Tensor): A 2D tensor of shape [N, 2] containing the (x, y) coordinates at which to extract pixel values.
    Returns:
        color_values (torch.Tensor): A 3D tensor of shape [batch, N, channel] containing the pixel values at the specified coordinates.
        coords (torch.Tensor): A 2D tensor of shape [N, 2] containing the normalized coordinates in the range [-1, 1].
    """
    # Normalize pixel coordinates to [-1, 1] range
    print("-----------------fetching_features_from_tensor----------------------")
    print("image_tensor shape:", image_tensor.shape)
    input_coords = input_coords.to(image_tensor.device)
    print("input_coords shape:", input_coords.shape)
    coords = input_coords / torch.tensor([image_tensor.shape[-2], image_tensor.shape[-1]],
                                         device=image_tensor.device).float()
    print("coords shape:", coords.shape)
    center_coords_normalized = torch.tensor([0.5, 0.5], device=image_tensor.device).float()
    print("center_coords_normalized shape:", center_coords_normalized.shape)
    coords = (center_coords_normalized - coords) * 2.0
    print("coords shape:", coords.shape)
    # Fetching the colour of the pixels in each coordinates
    batch_size = image_tensor.shape[0]
    print("batch_size:", batch_size)
    input_coords_expanded = input_coords.unsqueeze(0).expand(batch_size, -1, -1)
    print("input_coords_expanded shape:", input_coords_expanded.shape)
    y_coords = input_coords_expanded[..., 0].long()
    x_coords = input_coords_expanded[..., 1].long()
    print("y_coords shape:", y_coords.shape, "x_coords shape:", x_coords.shape)
    batch_indices = torch.arange(batch_size).view(-1, 1).to(input_coords.device)
    print("batch_indices shape:", batch_indices.shape)
    color_values = image_tensor[batch_indices, :, x_coords, y_coords]
    print("color_values shape:", color_values.shape, "coords shape:", coords.shape)
    print("-----------------fetching_features_from_tensor----------------------")
    return color_values, coords


def extract_patch(image, center, radius, padding_mode='constant'):
    """
    Extract a patch from an image with the specified center and radius.
    Args:
        image (torch.Tensor): Input image of shape [batch_size, channels, height, width].
        center (tuple): Coordinates (y, x) of the patch center.
        radius (int): Radius of the patch.
        padding_mode (str, optional): Padding mode, can be 'constant', 'reflect', 'replicate', or 'circular'. Default is 'constant'.

    Returns:
        torch.Tensor: Extracted patch of shape [batch_size, channels, 2 * radius, 2 * radius].
    """
    height, width = image.shape[-2:]

    # Convert center coordinates to integers
    center_y, center_x = int(round(center[0])), int(round(center[1]))

    # Calculate patch boundaries
    top = center_y - radius
    bottom = center_y + radius
    left = center_x - radius
    right = center_x + radius

    # Check if boundaries are out of image bounds
    top_padding = max(0, -top)
    bottom_padding = max(0, bottom - height)
    left_padding = max(0, -left)
    right_padding = max(0, right - width)

    # Pad the image
    padded_image = torch.nn.functional.pad(image, (left_padding, right_padding, top_padding, bottom_padding),
                                           mode=padding_mode)

    # Extract the patch
    patch = padded_image[..., top_padding:top_padding + 2 * radius, left_padding:left_padding + 2 * radius]

    return patch


@register('gaussian-splatter')
class GaussianSplatter(nn.Module):
    """A module that applies 2D Gaussian splatting to input features."""

    def __init__(self, encoder_spec, dec_spec, kernel_size, hidden_dim=256, unfold_row=7, unfold_column=7,
                 num_points=100):
        """
        Initialize the 2D Gaussian Splatter module.
        Args:
            kernel_size (int): The size of the kernel to convert rasterization.
            unfold_row (int): The number of points in the row dimension of the Gaussian grid.
            unfold_column (int): The number of points in the column dimension of the Gaussian grid.
        """
        super(GaussianSplatter, self).__init__()
        self.encoder = models.make(encoder_spec)
        self.feat, self.logits = None, None

        self.coef = nn.Conv2d(self.encoder.out_dim, hidden_dim, 3, padding=1)
        self.freq = nn.Conv2d(self.encoder.out_dim, hidden_dim, 3, padding=1)
        self.phase = nn.Linear(2, hidden_dim // 2, bias=False)
        self.dec = models.make(dec_spec, args={'in_dim': hidden_dim})

        # Key parameter in 2D Gaussian Splatter
        self.kernel_size = kernel_size
        self.row = unfold_row
        self.column = unfold_column
        self.num_points = num_points

        # Initialize Trainable Parameters
        sigma_x, sigma_y = torch.meshgrid(torch.linspace(0.2, 3.0, 10), torch.linspace(0.2, 3.0, 10))
        self.sigma_x = sigma_x.reshape(-1)
        self.sigma_y = sigma_y.reshape(-1)
        self.opacity = torch.sigmoid(torch.ones(self.num_points, 1, requires_grad=True))
        self.rho = torch.clamp(torch.zeros(self.num_points, 1, requires_grad=True), min=-1, max=1)
        self.sigma_x = nn.Parameter(self.sigma_x)  # Standard deviation in x-axis
        self.sigma_y = nn.Parameter(self.sigma_y)  # Standard deviation in y-axis
        self.opacity = nn.Parameter(self.opacity)  # Transparency of feature, shape=[num_points, 1]
        self.rho = nn.Parameter(self.rho)

    def weighted_gaussian_parameters(self, logits):
        """
        Computes weighted Gaussian parameters based on logits and the Gaussian kernel parameters (sigma_x, sigma_y, opacity).
        The logits tensor is used as a weight to compute a weighted sum of the Gaussian kernel parameters for each spatial
        location across the batch dimension. The resulting weighted parameters are then averaged across the batch dimension.
        Args:
            logits (torch.Tensor): Logits tensor of shape [batch, class, height, width].
        Returns:
            tuple: A tuple containing the weighted Gaussian parameters:
                - weighted_sigma_x (torch.Tensor): Tensor of shape [height * width] representing the weighted x-axis standard deviations.
                - weighted_sigma_y (torch.Tensor): Tensor of shape [height * width] representing the weighted y-axis standard deviations.
                - weighted_opacity (torch.Tensor): Tensor of shape [height * width] representing the weighted opacities.
        Description:
            This function computes weighted Gaussian parameters based on the input tensor, logits, and the provided Gaussian kernel parameters (sigma_x, sigma_y, and opacity). The logits tensor is used as a weight to compute a weighted sum of the Gaussian kernel parameters for each spatial location (height and width) across the batch dimension. The resulting weighted parameters are then averaged across the batch dimension, yielding tensors of shape [height * width] for the weighted sigma_x, sigma_y, and opacity.
        """
        batch_size, num_classes, height, width = logits.size()
        logits = logits.permute(0, 2, 3, 1)  # Reshape logits to [batch, height, width, class] = [100, 7, 7, 100]

        # Compute weighted sum of Gaussian parameters across class dimension
        weighted_sigma_x = (logits * self.sigma_x.unsqueeze(0).unsqueeze(0).unsqueeze(0)).sum(dim=-1)
        weighted_sigma_y = (logits * self.sigma_y.unsqueeze(0).unsqueeze(0).unsqueeze(0)).sum(dim=-1)
        weighted_opacity = (logits * self.opacity[:, 0].unsqueeze(0).unsqueeze(0).unsqueeze(0)).sum(dim=-1)
        weighted_rho = (logits * self.rho[:, 0].unsqueeze(0).unsqueeze(0).unsqueeze(0)).sum(dim=-1)
        print("-----------------weighted_gaussian_parameters----------------------")
        print("weighted_sigma_x shape:", weighted_sigma_x.shape, "weighted_sigma_y shape:", weighted_sigma_y.shape, "weighted_opacity shape:", weighted_opacity.shape, "weighted_rho shape:", weighted_rho.shape)

        # Reshape and average across batch dimension
        weighted_sigma_x = weighted_sigma_x.reshape(batch_size, -1).mean(dim=0)
        weighted_sigma_y = weighted_sigma_y.reshape(batch_size, -1).mean(dim=0)
        weighted_opacity = weighted_opacity.reshape(batch_size, -1).mean(dim=0)
        weighted_rho = weighted_rho.reshape(batch_size, -1).mean(dim=0)
        print("after reshape and mean")
        print("weighted_sigma_x shape:", weighted_sigma_x.shape, "weighted_sigma_y shape:", weighted_sigma_y.shape, "weighted_opacity shape:", weighted_opacity.shape, "weighted_rho shape:", weighted_rho.shape)
        print("-----------------weighted_gaussian_parameters end----------------------")
        return weighted_sigma_x, weighted_sigma_y, weighted_opacity, weighted_rho

    def gen_feat(self, inp):
        """Generate feature and logits by encoder."""
        self.inp = inp
        self.feat, self.logits = self.encoder(inp)
        self.feat_coord = make_coord(inp.shape[-2:], flatten=False).cuda().permute(2, 0, 1) \
            .unsqueeze(0).expand(inp.shape[0], 2, *inp.shape[-2:])
        return self.feat, self.logits

    def query_rgb(self, coord, scale, cell=None):
        """
        Continuous sampling through 2D Gaussian Splatting.
        Args:
            coord (torch.Tensor): [Batch, Sample_q, 2]. The normalized coordinates of HR space (of range [-1, 1]).
            cell (torch.Tensor): [Batch, Sample_q, 2]. The normalized cell size of HR space.
            scale (torch.Tensor): [Batch]. The magnification scale of super-resolution. (1, 4) during training.
        Returns:
            torch.Tensor: The output features after Gaussian splatting, of the same shape as the input.
        """
        # 1. Get LR feature and logits
        print('Encoder feature size:', self.feat.shape)
        feat, lr_feat, logits = self.feat[:, :8, :, :], self.feat[:, 8:, :, :], self.logits  # Channel decoupling
        print("2DGS feat shape:", feat.shape, "2DGS lr_feat shape:", lr_feat.shape, "2DGS logits shape:", logits.shape)
        feat_size, feat_device = feat.shape, feat.device

        # 2. Calculate the high-resolution image size
        scale = float(scale[0])
        print("Scale:", scale)
        hr_h = round(feat.shape[-2] * scale)  # shape: [batch size]
        hr_w = round(feat.shape[-1] * scale)
        print("HR size:", hr_h, hr_w)
        
        # 3. Unfold the feature / logits to many small patches to avoid extreme GPU memory consumption
        num_kernels_row = math.ceil(feat_size[-2] / self.row)
        num_kernels_column = math.ceil(feat_size[-1] / self.column)
        print("num_kernels_row:", num_kernels_row, "num_kernels_column:", num_kernels_column)
        upsampled_size = (num_kernels_row * self.row, num_kernels_column * self.column)
        print("upsampled_size:", upsampled_size)
        upsampled_inp = F.interpolate(feat, size=upsampled_size, mode='bicubic', align_corners=False)
        upsampled_logits = F.interpolate(logits, size=upsampled_size, mode='bicubic', align_corners=False)
        unfold = nn.Unfold(kernel_size=(self.row, self.column), stride=(self.row, self.column))
        unfolded_feature = unfold(upsampled_inp)
        unfolded_logits = unfold(upsampled_logits)
        print("unfolded_feature shape:", unfolded_feature.shape, "unfolded_logits shape:", unfolded_logits.shape)
        # Unfolded_feature dimension becomes [Batch, C*K*K, L], where L is the number of columns after unfolding
        L = unfolded_feature.shape[-1]
        print("L:", L)
        unfolded_feature_reshaped = unfolded_feature.transpose(1, 2). \
            reshape(feat_size[0] * L, feat_size[1], self.row, self.column)
        unfold_feat = unfolded_feature_reshaped  # shape: [num of patch * batch, channel, self.row, self.column]
        unfolded_logits_reshaped = unfolded_logits.transpose(1, 2). \
            reshape(logits.shape[0] * L, logits.shape[1], self.row, self.column)
        unfold_logits = unfolded_logits_reshaped  # shape: [num of patch * batch, channel, self.row, self.column]
        print("unfold_feat shape:", unfold_feat.shape, "unfold_logits shape:", unfold_logits.shape)
        
        # 4. Generate colors_(features) and coords_norm
        coords_ = generate_meshgrid(unfold_feat.shape[-2], unfold_feat.shape[-1])  # 메쉬 그리드 생성
        print("coords_ shape:", coords_.shape)
        num_LR_points = unfold_feat.shape[-2] * unfold_feat.shape[-1]  # 저해상도 포인트 수 계산
        print("num_LR_points:", num_LR_points)
        colors_, coords_norm = fetching_features_from_tensor(unfold_feat, coords_)  # 텐서에서 특징 추출
        print("colors_ shape:", colors_.shape, "coords_norm shape:", coords_norm.shape)

        # 5. Rasterization: Generating grid
        # 5.1. Spread Gaussian points over the whole feature map
        batch_size, channel, _, _ = unfold_feat.shape  # 배치 크기와 채널 수 가져오기
        weighted_sigma_x, weighted_sigma_y, weighted_opacity, weighted_rho = self.weighted_gaussian_parameters(
            unfold_logits)  # 가중치가 적용된 가우시안 파라미터 계산
        sigma_x = weighted_sigma_x.view(num_LR_points, 1, 1)  # 가우시안 x축 표준편차
        sigma_y = weighted_sigma_y.view(num_LR_points, 1, 1)  # 가우시안 y축 표준편차
        rho = weighted_rho.view(num_LR_points, 1, 1)  # 가우시안 상관계수
        print("sigma_x shape:", sigma_x.shape, "sigma_y shape:", sigma_y.shape, "rho shape:", rho.shape, "weighted_opacity shape:", weighted_opacity.shape)
        # 5.2. Gaussian expression
        covariance = torch.stack(
            [torch.stack([sigma_x ** 2 + 1e-5, rho * sigma_x * sigma_y], dim=-1),
             torch.stack([rho * sigma_x * sigma_y, sigma_y ** 2 + 1e-5], dim=-1)], dim=-2
        )  # 공분산 행렬 계산
        inv_covariance = torch.inverse(covariance).to(feat_device)  # 공분산 행렬의 역행렬 계산
        print("covariance shape:", covariance.shape, "inv_covariance shape:", inv_covariance.shape)
        
        # 5.3. Choosing a broad range for the distribution [-5,5] to avoid any clipping
        start = torch.tensor([-5.0], device=feat_device).view(-1, 1)  # 시작 범위 설정
        end = torch.tensor([5.0], device=feat_device).view(-1, 1)  # 끝 범위 설정
        print("start shape:", start.shape, "end shape:", end.shape)
        base_linspace = torch.linspace(0, 1, steps=self.kernel_size, device=feat_device)  # 선형 공간 생성
        ax_batch = start + (end - start) * base_linspace  # 배치 축 생성
        print("base_linspace shape:", base_linspace.shape, "ax_batch shape:", ax_batch.shape)
        # Expanding dims for broadcasting
        ax_batch_expanded_x = ax_batch.unsqueeze(-1).expand(-1, -1, self.kernel_size)  # x축 확장
        ax_batch_expanded_y = ax_batch.unsqueeze(1).expand(-1, self.kernel_size, -1)  # y축 확장
        print("ax_batch_expanded_x shape:", ax_batch_expanded_x.shape, "ax_batch_expanded_y shape:", ax_batch_expanded_y.shape)

        # 5.4. Creating a batch-wise meshgrid using broadcasting
        xx, yy = ax_batch_expanded_x, ax_batch_expanded_y  # 메쉬 그리드 생성
        xy = torch.stack([xx, yy], dim=-1)  # x, y 좌표 스택
        print("xy shape:", xy.shape)
        z = torch.einsum('b...i,b...ij,b...j->b...', xy, -0.5 * inv_covariance, xy)  # 가우시안 계산
        print("z shape:", z.shape)
        kernel = torch.exp(z) / (2 * torch.tensor(np.pi, device=feat_device) *
                     torch.sqrt(torch.det(covariance)).to(feat_device).view(num_LR_points, 1, 1))  # 커널 계산
        print("kernel shape:", kernel.shape)
        kernel_max_1, _ = kernel.max(dim=-1, keepdim=True)  # 첫 번째 최대값 찾기
        kernel_max_2, _ = kernel_max_1.max(dim=-2, keepdim=True)  # 두 번째 최대값 찾기
        kernel_normalized = kernel / kernel_max_2  # 커널 정규화
        kernel_reshaped = kernel_normalized.repeat(1, channel, 1).contiguous(). \
            view(num_LR_points * channel, self.kernel_size, self.kernel_size)  # 커널 재구성
        kernel_color = kernel_reshaped.unsqueeze(0).reshape(num_LR_points, channel, self.kernel_size, self.kernel_size)  # 커널 색상

        # 5.5. Adding padding to make kernel size equal to the image size
        pad_h = round(unfold_feat.shape[-2] * scale) - self.kernel_size  # 패딩 높이 계산
        pad_w = round(unfold_feat.shape[-1] * scale) - self.kernel_size  # 패딩 너비 계산
        print("pad_h:", pad_h, "pad_w:", pad_w)
        if pad_h < 0 or pad_w < 0:  # 패딩 크기 확인
            raise ValueError("Kernel size should be smaller or equal to the image size.")  # 오류 발생
        padding = (pad_w // 2, pad_w // 2 + pad_w % 2, pad_h // 2, pad_h // 2 + pad_h % 2)  # 패딩 값 설정
        kernel_color_padded = torch.nn.functional.pad(kernel_color, padding, "constant", 0)  # 패딩 적용
        print("padding: ", padding)
        print("kernel_color_padded shape:", kernel_color_padded.shape)

        # 5.6. Create a batch of 2D affine matrices
        b, c, h, w = kernel_color_padded.shape  # num_LR_points, channel, hr_h, hr_w
        theta = torch.zeros(batch_size, b, 2, 3, dtype=torch.float32, device=feat_device)  # 2D 아핀 행렬 생성
        theta[:, :, 0, 0] = 1.0  # x축 스케일 설정
        theta[:, :, 1, 1] = 1.0  # y축 스케일 설정
        theta[:, :, :, 2] = coords_norm  # 좌표 설정
        print("theta shape:", theta.shape)
        grid = F.affine_grid(theta.view(-1, 2, 3), size=[batch_size * b, c, h, w], align_corners=True).contiguous()  # 그리드 생성
        print("grid shape:", grid.shape)
        kernel_color_padded_expanded = kernel_color_padded.repeat(batch_size, 1, 1, 1).contiguous()  # 패딩 확장
        print("kernel_color_padded_expanded shape:", kernel_color_padded_expanded.shape)
        kernel_color_padded_translated = F.grid_sample(kernel_color_padded_expanded.contiguous(), grid.contiguous(),
                                   align_corners=True)  # 그리드 샘플링
        kernel_color_padded_translated = kernel_color_padded_translated.view(batch_size, b, c, h, w)  # 재구성
        print("kernel_color_padded_translated shape:", kernel_color_padded_translated.shape)

        # 6. Apply Gaussian splatting
        # colors_.shape = [batch, num_LR_points, channel], colors.shape = [batch, num_LR_points, channel]
        colors = colors_ * weighted_opacity.to(feat_device).unsqueeze(-1).expand(batch_size, -1, -1)  # 색상 적용
        print("colors shape:", colors.shape)
        color_values_reshaped = colors.unsqueeze(-1).unsqueeze(-1)  # 색상 재구성
        print("color_values_reshaped shape:", color_values_reshaped.shape)
        final_image_layers = color_values_reshaped * kernel_color_padded_translated  # 최종 이미지 레이어 생성
        final_image = final_image_layers.sum(dim=1)  # 최종 이미지 합산
        final_image = torch.clamp(final_image, 0, 1)  # 이미지 클램프
        print("final_image shape:", final_image.shape)

        # 7. Fold the input back to the original size
        # Calculate the number of kernels needed to cover each dimension.
        kernel_h, kernel_w = round(self.row * scale), round(self.column * scale)  # 커널 크기 계산
        print("kernel_h:", kernel_h, "kernel_w:", kernel_w)
        fold = nn.Fold(output_size=(kernel_h * num_kernels_row, kernel_w * num_kernels_column),
                   kernel_size=(kernel_h, kernel_w), stride=(kernel_h, kernel_w))  # 폴드 설정
        final_image = final_image.reshape(feat_size[0], L, feat_size[1] * kernel_h * kernel_w).transpose(1, 2)  # 이미지 재구성
        print("final_image shape:", final_image.shape)
        final_image = fold(final_image)  # 폴드 적용
        print("final_image shape after folding:", final_image.shape)
        final_image = F.interpolate(final_image, size=(hr_h, hr_w), mode='bicubic', align_corners=False)  # 이미지 보간
        print("final_image shape after interpolation:", final_image.shape)
        # Combine channel
        lr_feat = F.interpolate(lr_feat, size=(hr_h, hr_w), mode='bicubic', align_corners=False)  # 저해상도 특징 보간
        print("lr_feat shape after interpolation:", lr_feat.shape)
        final_image = torch.concat((final_image, lr_feat), dim=1)  # 채널 결합
        print("final_image shape after concatenation:", final_image.shape)
        print("-----------------query_rgb end----------------------")
        print("-----------------augmentation start----------------------")
        # 8. Augmentation (Useful for improving out-of-distribution scale performance)
        coef = self.coef(final_image)  # 계수 계산
        freq = self.freq(final_image)  # 주파수 계산
        feat_coord = self.feat_coord  # 특징 좌표 설정
        print("coef shape:", coef.shape, "freq shape:", freq.shape, "feat_coord shape:", feat_coord.shape, "hr coord shape:", coord.shape)
        coord_ = coord.clone()  # 좌표 복제
        q_coef = F.grid_sample(coef, coord_.flip(-1).unsqueeze(1), mode='nearest', align_corners=False)[:, :, 0, :] \
            .permute(0, 2, 1)  # 계수 샘플링
        q_freq = F.grid_sample(freq, coord_.flip(-1).unsqueeze(1), mode='nearest', align_corners=False)[:, :, 0, :] \
            .permute(0, 2, 1)  # 주파수 샘플링
         
        feat_coord = feat_coord.to(coord_.device)  # 특징 좌표 디바이스 설정
        q_coord = F.grid_sample(feat_coord, coord_.flip(-1).unsqueeze(1), mode='nearest', align_corners=False)[:, :, 0,
              :] \
            .permute(0, 2, 1)  # 좌표 샘플링
        print("q_coef shape:", q_coef.shape, "q_freq shape:", q_freq.shape, "q_coord shape:", q_coord.shape)   
        
        rel_coord = coord - q_coord  # 상대 좌표 계산
        rel_coord[:, :, 0] *= feat.shape[-2]  # 상대 좌표 x축 계산
        rel_coord[:, :, 1] *= feat.shape[-1]  # 상대 좌표 y축 계산
        rel_cell = cell.clone()  # 셀 복제
        rel_cell[:, :, 0] *= feat.shape[-2]  # 상대 셀 x축 계산
        rel_cell[:, :, 1] *= feat.shape[-1]  # 상대 셀 y축 계산
        bs, q = coord.shape[:2]  # 배치 크기와 쿼리 수 설정
        q_freq = torch.stack(torch.split(q_freq, 2, dim=-1), dim=-1)  # 주파수 스택
        q_freq = torch.mul(q_freq, rel_coord.unsqueeze(-1))  # 주파수 곱셈
        q_freq = torch.sum(q_freq, dim=-2)  # 주파수 합산
        q_freq += self.phase(rel_cell.view((bs * q, -1))).view(bs, q, -1)  # 주파수 계산
        q_freq = torch.cat((torch.cos(np.pi * q_freq), torch.sin(np.pi * q_freq)), dim=-1)  # 주파수 결합

        inp = torch.mul(q_coef, q_freq)  # 입력 계산
        print("q coef shape: ", q_coef.shape, "q freq shape: ", q_freq.shape, "inp shape: ", inp.shape)
        pred = self.dec(inp.contiguous().view(bs * q, -1)).view(bs, q, -1)  # 예측값 계산

        return pred

    def forward(self, inp, coord, scale, cell=None):
        self.gen_feat(inp)
        return self.query_rgb(coord, scale, cell)


if __name__ == '__main__':
    # A simple example of implementing class GaussianSplatter
    model = GaussianSplatter(encoder_spec={"name": "edsr-baseline", "args": {"no_upsampling": True}},
                             dec_spec={"name": "mlp", "args": {"out_dim": 3, "hidden_list": [256, 256, 256, 256]}},
                             kernel_size=3)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input = torch.rand(1, 3, 64, 64)
    # input = input.to(device)
    sr_scale = 2
    hr_coord, hr_rgb = to_pixel_samples(
        F.interpolate(input, size=(round(input.shape[-2] * sr_scale), round(input.shape[-1] * sr_scale)),
                      mode='bicubic', align_corners=False))
    v0_x, v1_x, v0_y, v1_y = -1, 1, -1, 1
    nx, ny = round(input.shape[-2] * sr_scale), round(input.shape[-1] * sr_scale)

    x = ((hr_coord[..., 0] - v0_x) / (v1_x - v0_x) * 2 * (nx - 1) / 2).round().long()
    y = ((hr_coord[..., 1] - v0_y) / (v1_y - v0_y) * 2 * (ny - 1) / 2).round().long()
    restored_coords = torch.stack([x, y], dim=-1)

    sample_lst = np.random.choice(len(hr_coord), 2304, replace=False)
    hr_coord = hr_coord[sample_lst]
    hr_rgb = hr_rgb[sample_lst]
    cell_ = torch.ones_like(hr_coord.unsqueeze(0))
    cell_[:, 0] *= 2 / nx
    cell_[:, 1] *= 2 / ny
    sr_scale = 2 * torch.ones(1)
    # print(f"Input device: {input.device}")
    print("input size: ", input.shape, "hr_coord size: ", hr_coord.shape, "sr_scale size: ", sr_scale.shape, "cell_ size: ", cell_.shape)
    print("model output shape: ", model(input, hr_coord.unsqueeze(0), sr_scale, cell_).shape)


File not found: models/weights/edsr_baseline_x2-1bc95232.pt
input size:  torch.Size([1, 3, 64, 64]) hr_coord size:  torch.Size([2304, 2]) sr_scale size:  torch.Size([1]) cell_ size:  torch.Size([1, 2304, 2])
Encoder feature size: torch.Size([1, 64, 64, 64])
2DGS feat shape: torch.Size([1, 8, 64, 64]) 2DGS lr_feat shape: torch.Size([1, 56, 64, 64]) 2DGS logits shape: torch.Size([1, 100, 64, 64])
Scale: 2.0
HR size: 128 128
num_kernels_row: 10 num_kernels_column: 10
upsampled_size: (70, 70)
unfolded_feature shape: torch.Size([1, 392, 100]) unfolded_logits shape: torch.Size([1, 4900, 100])
L: 100
unfold_feat shape: torch.Size([100, 8, 7, 7]) unfold_logits shape: torch.Size([100, 100, 7, 7])
coords_ shape: torch.Size([49, 2])
num_LR_points: 49
-----------------fetching_features_from_tensor----------------------
image_tensor shape: torch.Size([100, 8, 7, 7])
input_coords shape: torch.Size([49, 2])
coords shape: torch.Size([49, 2])
center_coords_normalized shape: torch.Size([2])
coords shape

c:\Workspace\2d_gaussian_SR\models\edsr.py:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(args.pretrained_path)


In [28]:
import torch.nn as nn

from models import register

class MLP(nn.Module):

    def __init__(self, in_dim, out_dim, hidden_list, act='gelu'):
        super().__init__()
        if act is None:
            self.act = None
        elif act.lower() == 'relu':
            self.act = nn.ReLU()
        elif act.lower() == 'gelu':
            self.act = nn.GELU()
        else:
            assert False, f'activation {act} is not supported'
        layers = []
        lastv = in_dim
        for hidden in hidden_list:
            layers.append(nn.Linear(lastv, hidden))
            if self.act:
                layers.append(self.act)
            lastv = hidden
        layers.append(nn.Linear(lastv, out_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        shape = x.shape[:-1]
        x = self.layers(x.view(-1, x.shape[-1]))
        return x.view(*shape, -1)


# Create an instance of the MLP class
mlp_model = MLP(in_dim=256, out_dim=3, hidden_list=[256, 256, 256, 256], act='gelu')

# Print the structure of the MLP model
print(mlp_model)

MLP(
  (act): GELU(approximate='none')
  (layers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): GELU(approximate='none')
    (8): Linear(in_features=256, out_features=3, bias=True)
  )
)


In [39]:
import torch


# Create two 2x2 matrices
matrix1 = torch.tensor([[1, 2], [3, 4]])
matrix2 = torch.tensor([[4, 3], [2, 1]])

print("Matrix 1:")
print(matrix1)

print("Matrix 2:")
print(matrix2)

mul = torch.mul(matrix1, matrix2)
print("Matrix Multiplication:")
print(mul)

Matrix 1:
tensor([[1, 2],
        [3, 4]])
Matrix 2:
tensor([[4, 3],
        [2, 1]])
Matrix Multiplication:
tensor([[4, 6],
        [6, 4]])
